In [1]:
!pip install ultralytics kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.1 MB/s eta 0:00:00


In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("phucthaiv02/butterfly-image-classification")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'butterfly-image-classification' dataset.
Path to dataset files: /kaggle/input/butterfly-image-classification


In [14]:
# Установка (если нужно)
!pip install kagglehub -q

import kagglehub
import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 1. Загрузка датасета
print("Загрузка датасета")
path = kagglehub.dataset_download("phucthaiv02/butterfly-image-classification")
print("Путь к датасету:", path)

dataset_root = Path(path)

# 2. Собираем ВСЕ изображения из корневых папок-классов
all_images = []
all_labels = []

# Проходим по всем подпапкам в корне — каждая = отдельный класс
for class_dir in dataset_root.iterdir():
    if class_dir.is_dir():
        class_name = class_dir.name
        # Ищем изображения внутри папки класса
        for img_path in class_dir.glob("*"):
            if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
                all_images.append(str(img_path))
                all_labels.append(class_name)

print(f" Всего изображений найдено: {len(all_images)}")
print(f" Классов: {len(set(all_labels))}")

if len(all_images) == 0:
    raise FileNotFoundError(" Не найдено ни одного изображения")

# 3. Создаём новую структуру 70/15/15
new_dataset_root = Path("/content/butterfly_dataset_70_15_15")
shutil.rmtree(new_dataset_root, ignore_errors=True)
(new_dataset_root / "train").mkdir(parents=True)
(new_dataset_root / "val").mkdir(parents=True)
(new_dataset_root / "test").mkdir(parents=True)

# 4. Разбиваем сначала на train (70%) и остальное (30%)
X_train, X_rest, y_train, y_rest = train_test_split(
    all_images, all_labels,
    test_size=0.3,
    random_state=42,
    stratify=all_labels
)

# Затем делим "остальное" пополам → 15% val, 15% test
X_val, X_test, y_val, y_test = train_test_split(
    X_rest, y_rest,
    test_size=0.5,
    random_state=42,
    stratify=y_rest
)

print(f"\nРазмеры:")
print(f"  train: {len(X_train)}")
print(f"  val:   {len(X_val)}")
print(f"  test:  {len(X_test)}")

# 5. Функция копирования
def copy_images(images, labels, split_name):
    split_dir = new_dataset_root / split_name
    for img_path, label in tqdm(zip(images, labels), total=len(images), desc=f"Копирование → {split_name}"):
        class_dir = split_dir / label
        class_dir.mkdir(exist_ok=True)
        shutil.copy2(img_path, class_dir / Path(img_path).name)

# 6. Копируем
copy_images(X_train, y_train, "train")
copy_images(X_val, y_val, "val")
copy_images(X_test, y_test, "test")

# 7. Создаём YAML для YOLO
classes = sorted(set(all_labels))
yaml_content = f"""path: /content/butterfly_dataset_70_15_15
train: train
val: val
test: test
names: {classes}
"""

with open("/content/butterfly.yaml", "w") as f:
    f.write(yaml_content)

print("\n Датасет успешно пересобран в формате 70/15/15!")
print(" Путь:", new_dataset_root)
print(" YAML:", "/content/butterfly.yaml")

Загрузка датасета
Using Colab cache for faster access to the 'butterfly-image-classification' dataset.
Путь к датасету: /kaggle/input/butterfly-image-classification
 Всего изображений найдено: 9285
 Классов: 2

Размеры:
  train: 6499
  val:   1393
  test:  1393


Копирование → test: 100%|██████████| 1393/1393 [00:02<00:00, 516.46it/s]


 Датасет успешно пересобран в формате 70/15/15!
 Путь: /content/butterfly_dataset_70_15_15
 YAML: /content/butterfly.yaml


In [15]:
# 1. Установка зависимостей
!pip install kagglehub ultralytics -q

# 2. Импорты
import kagglehub
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from ultralytics import YOLO

# Часть 1: Подготовка датасета

print(" Загрузка датасета...")
path = kagglehub.dataset_download("phucthaiv02/butterfly-image-classification")
dataset_root = Path(path)
print("Путь к исходному датасету:", dataset_root)

# Сбор всех изображений и меток
SUPPORTED_EXTS = {'.jpg', '.jpeg', '.png', '.bmp'}
all_images = []
all_labels = []

for class_dir in dataset_root.iterdir():
    if class_dir.is_dir():
        class_name = class_dir.name
        for img_path in class_dir.iterdir():
            if img_path.suffix.lower() in SUPPORTED_EXTS:
                all_images.append(str(img_path))
                all_labels.append(class_name)

print(f"Всего изображений: {len(all_images)}")
print(f"Классов: {len(set(all_labels))}")

if not all_images:
    raise FileNotFoundError("Не найдено ни одного изображения!")

# Создание новой структуры 70/15/15
new_dataset_root = Path("/content/butterfly_dataset_70_15_15")
shutil.rmtree(new_dataset_root, ignore_errors=True)

for split in ["train", "val", "test"]:
    (new_dataset_root / split).mkdir(parents=True, exist_ok=True)

# Стратифицированное разбиение
X_train, X_rest, y_train, y_rest = train_test_split(
    all_images, all_labels,
    test_size=0.3,
    random_state=42,
    stratify=all_labels
)

X_val, X_test, y_val, y_test = train_test_split(
    X_rest, y_rest,
    test_size=0.5,
    random_state=42,
    stratify=y_rest
)

print(f"\nРазмеры:")
print(f"  train: {len(X_train)}")
print(f"  val:   {len(X_val)}")
print(f"  test:  {len(X_test)}")

# Функция копирования
def copy_images(images, labels, split_name):
    split_dir = new_dataset_root / split_name
    for img_path, label in tqdm(zip(images, labels), total=len(images), desc=f"Копирование → {split_name}"):
        class_dir = split_dir / label
        class_dir.mkdir(exist_ok=True)
        shutil.copy2(img_path, class_dir / Path(img_path).name)

# Копируем данные
copy_images(X_train, y_train, "train")
copy_images(X_val, y_val, "val")
copy_images(X_test, y_test, "test")

print("Путь:", new_dataset_root)

# Часть 2: Обучение YOLOv8-cls

# Проверка структуры
train_dir = new_dataset_root / "train"
val_dir = new_dataset_root / "val"
if not (train_dir.exists() and val_dir.exists()):
    raise FileNotFoundError(" Папки 'train' или 'val' отсутствуют!")

# Получение списка классов (необязательно для обучения, но полезно для логов)
classes = sorted([d.name for d in train_dir.iterdir() if d.is_dir()])
print(f"\n Найдено классов: {len(classes)}")
print(f"Примеры: {classes[:5]}")

# Загрузка предобученной модели классификации
print("\n Загрузка модели yolov8n-cls.pt...")
model = YOLO("yolov8n-cls.pt")

# Обучение — ПЕРЕДАЁМ ПУТЬ К ПАПКЕ, НЕ К YAML!
print("\n Запуск обучения...")
results = model.train(
    data=str(new_dataset_root),  # ← ВАЖНО: это директория, не файл!
    epochs=30,
    imgsz=224,
    batch=32,  # уменьшите при нехватке памяти GPU
    name="butterfly_yolov8_cls",
    device=0 if next(model.parameters()).is_cuda else "cpu",
    patience=10,
    exist_ok=True
)

# Валидация на val
print("\n Валидация на val...")
val_metrics = model.val()
print(f" Val Top-1 Accuracy: {val_metrics.top1:.4f}")
print(f" Val Top-5 Accuracy: {val_metrics.top5:.4f}")

# Валидация на test (если есть)
test_dir = new_dataset_root / "test"
if test_dir.exists() and any(test_dir.iterdir()):
    print("\n Валидация на test...")
    test_metrics = model.val(data=str(new_dataset_root), split="test")
    print(f" Test Top-1 Accuracy: {test_metrics.top1:.4f}")
else:
    print("\n Тестовая выборка отсутствует или пуста.")

# Пример предсказания
print("\n Пример предсказания...")
sample_img = None
for class_name in classes[:3]:
    class_val_path = val_dir / class_name
    if class_val_path.exists():
        for f in class_val_path.iterdir():
            if f.suffix.lower() in SUPPORTED_EXTS:
                sample_img = f
                true_class = class_name
                break
    if sample_img:
        break

if sample_img:
    result = model(str(sample_img))
    probs = result[0].probs
    pred_class = classes[int(probs.top1)]
    conf = float(probs.top1conf)
    print(f"Истинный класс: {true_class}")
    print(f"Предсказание: {pred_class} (уверенность: {conf:.4f})")
else:
    print(" Не удалось найти изображение для примера.")

print("\n Обучение завершено! Модель сохранена в runs/classify/butterfly_yolov8_cls/")

 Загрузка датасета...
Using Colab cache for faster access to the 'butterfly-image-classification' dataset.
Путь к исходному датасету: /kaggle/input/butterfly-image-classification
Всего изображений: 9285
Классов: 2

Размеры:
  train: 6499
  val:   1393
  test:  1393


Копирование → test: 100%|██████████| 1393/1393 [00:01<00:00, 1226.35it/s]


Путь: /content/butterfly_dataset_70_15_15

 Найдено классов: 2
Примеры: ['test', 'train']

 Загрузка модели yolov8n-cls.pt...

 Запуск обучения...
New https://pypi.org/project/ultralytics/8.3.216 available   Update with 'pip install -U ultralytics'
Ultralytics 8.3.215 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/butterfly_dataset_70_15_15, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, m